In [15]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import datetime

In [3]:
if tf.__version__.startswith("2."):
  import tensorflow.compat.v1 as tf
  tf.compat.v1.disable_v2_behavior()
  tf.compat.v1.disable_eager_execution()
  print("Enabled compatitility to tf1.x")

Instructions for updating:
non-resource variables are not supported in the long term
Enabled compatitility to tf1.x


In [4]:
#cargamos los datos del proyecto
data = np.load('proyecto_training_data.npy')
print('filas: ',data.shape[0])
print('columnas: ',data.shape[1])
n = len(data)
print(n)

filas:  1460
columnas:  6
1460


In [ ]:
df = pd.DataFrame(data,columns=["sale_price","overall_qual","square_feet","total_room","year_build","lot_frontage"])
df.head()
df.to_csv(index=False)

In [6]:
#dividiendo data
n = len(df)
y_train = np.array(df[:int(n*0.8)]["sale_price"]/1000)
y_test = np.array(df[len(y_train):n]["sale_price"]/1000)
x_train = np.array(df[:int(n*0.8)]["overall_qual"])
x_test = np.array(df[len(y_train):n]["overall_qual"])

In [7]:
#obtenermos los parametros del modelo, inicializados aleatoriamente
def reset():
    tf.reset_default_graph()
    m = tf.Variable(np.random.randn(), name = "m") 
    b = tf.Variable(np.random.randn(), name = "b")
    return m,b

In [8]:
#obtenie las predicciones del modelo
def model(m,b,x):
    return tf.add(tf.multiply(m,x),b)

In [9]:
#función de error para el modelo
def error(y,y_hat):
  return 1/2*tf.reduce_mean(tf.math.square(y - y_hat))

In [10]:
#función para obtener los gradientes
def gradient_descent(tensor_y,y_hat,m,b):
    return tf.gradients(error(tensor_y,y_hat),[m,b])

In [11]:
def training(X,Y,epochs,print_each,lr,m,b):
  y_hat = model(m,b,X)
  get_gradient = gradient_descent(Y,y_hat,m,b)
  ma = tf.assign(m, m - lr*get_gradient[0])
  ba = tf.assign(b, b - lr*get_gradient[1])
  update = tf.group(ma,ba)
  err = error(Y,y_hat)
  first_summary = tf.summary.scalar('error', err)
  with tf.train.MonitoredSession() as session:
      feed_dict = {X:x_train, Y:y_train}
      writer = tf.summary.FileWriter('./logs/'+datetime.datetime.now().strftime("%Y%m%d")+'lr_'+str(lr), session.graph)
      for i in range(epochs):
          session.run(get_gradient,feed_dict=feed_dict)
          session.run(update,feed_dict=feed_dict)
          summary = session.run(first_summary,feed_dict)
          writer.add_summary(summary, (i+1))

In [12]:
lr = 0.009
m,b = reset()

tensor_x = tf.placeholder(tf.float32,[len(x_train)],"tensor_x")
tensor_y = tf.placeholder(tf.float32,[len(x_train)],"tensor_x")

training(tensor_x,tensor_y,15000,3000,lr,m,b)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [21]:
%tensorboard --logdir logs

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.